In [63]:
# https://www.gcptutorials.com/post/how-to-get-weights-of-layers-in-tensorflow
# https://datascience.stackexchange.com/questions/85409/getting-nn-weights-for-every-batch-epoch-from-keras-model

In [45]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers



In [46]:
# Generate data

start, stop = 1,100
cnt = stop - start + 1
xs = np.linspace(start, stop, num = cnt)
b,k = 1,2
ys = np.array([k*x + b for x in xs])



In [52]:
# Simple model with one feature and one unit for regression task

model = keras.Sequential([
    layers.Dense(units=12, input_shape=[1], activation='relu'),
    layers.Dense(units=128, input_shape=[1], activation='relu'),
    layers.Dense(units=1, input_shape=[1], activation='relu')
])
model.compile(loss='mae', optimizer='adam')
batch_size = int(cnt / 5)
epochs = 80

In [50]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(6, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

In [53]:
checkpoint_filepath = './hdf5/checkpoint-{epoch:02d}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq ='epoch', # 1 for every batch
    save_best_only=False
)



In [54]:
# Train model

history = model.fit(xs, ys, batch_size=batch_size, epochs=epochs, 
                    callbacks=[model_checkpoint_callback])

Epoch 1/80
5/5 [==============================] - 0s 1ms/step - loss: 95.2379
Epoch 2/80
5/5 [==============================] - 0s 2ms/step - loss: 91.2242
Epoch 3/80
5/5 [==============================] - 0s 1ms/step - loss: 87.3436
Epoch 4/80
5/5 [==============================] - 0s 1ms/step - loss: 83.5259
Epoch 5/80
5/5 [==============================] - 0s 2ms/step - loss: 79.7353
Epoch 6/80
5/5 [==============================] - 0s 1ms/step - loss: 75.6827
Epoch 7/80
5/5 [==============================] - 0s 1ms/step - loss: 71.4746
Epoch 8/80
5/5 [==============================] - 0s 1ms/step - loss: 66.8123
Epoch 9/80
5/5 [==============================] - 0s 1ms/step - loss: 61.9225
Epoch 10/80
5/5 [==============================] - 0s 1ms/step - loss: 56.5916
Epoch 11/80
5/5 [==============================] - 0s 3ms/step - loss: 50.8747
Epoch 12/80
5/5 [==============================] - 0s 2ms/step - loss: 44.7368
Epoch 13/80
5/5 [==============================] - 0s 0s/step

In [10]:
w, b = model.weights
print("Weights: \n {} \n Bias: \n {}".format(w,b))



Weights: 
 <tf.Variable 'module_wrapper/dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.69388044]], dtype=float32)> 
 Bias: 
 <tf.Variable 'module_wrapper/dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.39999834], dtype=float32)>


In [8]:
# Functions to read weights from h5 file
import h5py

def getH5Keys(fileName):

    keys = []
    with h5py.File(fileName, mode='r') as f:
        for key in f:
            keys.append(key)

    return keys

def isGroup(obj):
    if isinstance(obj, h5py.Group):
        return True
    else:
        return False

def isDataset(obj):
    if isinstance(obj, h5py.Dataset):
        return True
    else:
        return False

def getDataSetsFromGroup(datasets, obj):
    if isGroup(obj):
        for key in obj:
            x = obj[key]
            getDataSetsFromGroup(datasets, x)
    else:
        datasets.append(obj)


def getWeightsForLayer(layerName, fileName):

    weights = []
    with h5py.File(fileName, mode='r') as f:
        for key in f:
            if layerName in key:
                obj = f[key]
                datasets = []
                getDataSetsFromGroup(datasets, obj)

                for dataset in datasets:
                    w = np.array(dataset)
                    weights.append(w)

    return weights

In [62]:
checkpoint_path = './hdf5/checkpoint-10.hdf5'
layers = getH5Keys(checkpoint_path)
firstLayer = layers[1]
print(layers) # ['dense']

weights = getWeightsForLayer(firstLayer, checkpoint_path)
for w in weights:
    print(w.shape)
print(weights)


['module_wrapper_4', 'module_wrapper_5', 'module_wrapper_6']
(128,)
(12, 128)
[array([ 0.05292301,  0.        ,  0.        , -0.04749306,  0.        ,
       -0.04234662,  0.        , -0.04213697,  0.        , -0.01821099,
        0.        ,  0.05441388,  0.05253087,  0.05335314, -0.02333801,
       -0.04810073,  0.0532589 ,  0.        ,  0.        , -0.00500865,
        0.05159298,  0.        , -0.04540244, -0.00749066,  0.05578383,
        0.05513497,  0.02916982,  0.        ,  0.        ,  0.05271865,
        0.05221001,  0.        ,  0.        ,  0.05181132,  0.05134611,
        0.        , -0.0459338 ,  0.05282699,  0.        ,  0.        ,
        0.0562271 ,  0.05439966,  0.        ,  0.        ,  0.        ,
       -0.01690347,  0.        ,  0.05854054,  0.        , -0.04482976,
       -0.0471217 , -0.01578558,  0.05172238,  0.        , -0.01820122,
        0.05299278, -0.02303774,  0.        ,  0.00264466, -0.04832368,
        0.        , -0.02059515,  0.05146533, -0.02630404

In [60]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper_4 (ModuleWrap (20, 12)                  24        
_________________________________________________________________
module_wrapper_5 (ModuleWrap (20, 128)                 1664      
_________________________________________________________________
module_wrapper_6 (ModuleWrap (20, 1)                   129       
Total params: 1,817
Trainable params: 1,817
Non-trainable params: 0
_________________________________________________________________


In [29]:
weights = getWeightsForLayer(firstLayer, checkpoint_path)
for w in weights:
    print(w.shape)
print(weights)

(1,)
(1, 1)
[array([0.8099931], dtype=float32), array([[1.1013573]], dtype=float32)]


In [12]:
weights_dict = {}
weight_callback = tf.keras.callbacks.LambdaCallback \
( on_epoch_end=lambda epoch, logs:  weights_dict.update({epoch:model.get_weights()}))



In [13]:
# Train model
history = model.fit(xs, ys, batch_size=batch_size, epochs=epochs, 
                    callbacks=[weight_callback])



Epoch 1/80
5/5 [==============================] - 0s 957us/step - loss: 66.4561
Epoch 2/80
5/5 [==============================] - 0s 1ms/step - loss: 66.2068
Epoch 3/80
5/5 [==============================] - 0s 2ms/step - loss: 65.9394
Epoch 4/80
5/5 [==============================] - 0s 1ms/step - loss: 65.6836
Epoch 5/80
5/5 [==============================] - 0s 2ms/step - loss: 65.4381
Epoch 6/80
5/5 [==============================] - 0s 990us/step - loss: 65.1785
Epoch 7/80
5/5 [==============================] - 0s 2ms/step - loss: 64.9162
Epoch 8/80
5/5 [==============================] - 0s 1ms/step - loss: 64.6734
Epoch 9/80
5/5 [==============================] - 0s 2ms/step - loss: 64.4163
Epoch 10/80
5/5 [==============================] - 0s 1ms/step - loss: 64.1527
Epoch 11/80
5/5 [==============================] - 0s 1ms/step - loss: 63.9010
Epoch 12/80
5/5 [==============================] - 0s 3ms/step - loss: 63.6385
Epoch 13/80
5/5 [==============================] - 0s 1ms

In [14]:
print(weights_dict[0])


[array([[0.6988332]], dtype=float32), array([0.40499827], dtype=float32)]


In [21]:
epoch = 4
print("*** Epoch: ", epoch, "\nWeight: ", weights_dict[0][0][0], " bias: ", weights_dict[1][0])


*** Epoch:  4 
Weight:  [0.6988332]  bias:  [[0.70384777]]
